In [ ]:
#!pip install py2neo
#!pip install neo4j
#!pip install spacy
#!pip install python-dateutil==2.1
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
!pip install crosslingual-coreference spacy-transformers==1.1.5 wikipedia neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz (15.9 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached crosslingual_coreference-0.2.8-py3-none-any.whl (11 kB)
  Using cached spacy_transformers-1.1.5-py2.py3-none-any.whl (51 kB)
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
     |████████████████████████████████| 173 kB 5.0 MB/s 
  Using cached spacy_alignments-0.8.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached transformers-4.17.0-py3-none-any.whl (3.8 MB)
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
  Using cached sacremoses-0.0.53.tar.gz (880 kB)
  Using cached tokenizers-0.13.1-cp37-cp37m-manylinux_2_17_x8

In [ ]:
import requests
import json
import csv
import io
import spacy
import crosslingual_coreference
from spacy import Language
from spacy.tokens import Doc, Span
from typing import List
from py2neo import Graph
from neo4j import GraphDatabase, basic_auth

ModuleNotFoundError: ignored

In [ ]:
def call_wiki_api(item):
  try:
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
    data = requests.get(url).json()
    # Return the first id (Could upgrade this in the future)
    return data['search'][0]['id']
  except:
    return 'id-less'

def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})

    return triplets

'''
@Language.factory(
    "rebel",
    requires=["doc.sents"],
    assigns=["doc._.rel"],
    default_config={
        "model_name": "Babelscape/rebel-large",
        "device": 0,
    },
)
'''
class RebelComponent:
    def __init__(
        self,
        nlp,
        name,
        model_name: str,
        device: int,
    ):
        assert model_name is not None, ""
        self.triplet_extractor = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
        self.entity_mapping = {}
        # Register custom extension on the Doc
        if not Doc.has_extension("rel"):
          Doc.set_extension("rel", default={})

    def get_wiki_id(self, item: str):
        mapping = self.entity_mapping.get(item)
        if mapping:
          return mapping
        else:
          res = call_wiki_api(item)
          self.entity_mapping[item] = res
          return res

    
    def _generate_triplets(self, sent: Span) -> List[dict]:
          output_ids = self.triplet_extractor(sent.text, return_tensors=True, return_text=False)[0]["generated_token_ids"]["output_ids"]
          extracted_text = self.triplet_extractor.tokenizer.batch_decode(output_ids[0])
          extracted_triplets = extract_triplets(extracted_text[0])
          return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        for triplet in triplets:

            # Remove self-loops (relationships that start and end at the entity)
            if triplet['head'] == triplet['tail']:
                continue

            # Use regex to search for entities
            head_span = re.search(triplet["head"], doc.text)
            tail_span = re.search(triplet["tail"], doc.text)

            # Skip the relation if both head and tail entities are not present in the text
            # Sometimes the Rebel model hallucinates some entities
            if not head_span or not tail_span:
              continue

            index = hashlib.sha1("".join([triplet['head'], triplet['tail'], triplet['type']]).encode('utf-8')).hexdigest()
            if index not in doc._.rel:
                # Get wiki ids and store results
                doc._.rel[index] = {"relation": triplet["type"], "head_span": {'text': triplet['head'], 'id': self.get_wiki_id(triplet['head'])}, "tail_span": {'text': triplet['tail'], 'id': self.get_wiki_id(triplet['tail'])}}

    def __call__(self, doc: Doc) -> Doc:
        for sent in doc.sents:
            sentence_triplets = self._generate_triplets(sent)
            self.set_annotations(doc, sentence_triplets)
        return doc

In [ ]:
def SubmitPMIDList(Pmid,Format,Bioconcept):

    # json = {"Pmids": Pmid}

    json = {}

    #
    # load pmids
    #
    with io.open(Inputfile,'r',encoding="utf-8") as file_input:
        json = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
    if Bioconcept != "": 
        json["concepts"] = Bioconcept.split(",")
    
    print(json)
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/" + Format , json = json)
    
    res = r.json()
    print(res)

    pmid = res['id']
    res0 = res['passages'][0]['annotations']
    res1 = res['passages'][1]['annotations']
    
    result = {}
    result = res0 + res1
    
    for i in result:
        i['Pmid'] = Pmid
    return result
    
    if r.status_code != 200 :
        print ("[Error]: HTTP code "+ str(r.status_code))
    else:
        return result

In [ ]:
Inputfile = "./pmid"
Format = "biocjson"
Bioconcept = ""
res_json=[]

In [ ]:
with io.open(Inputfile,'r',encoding="utf-8") as file_input:
    pmidlist = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
for i in pmidlist['pmids']:
    res_json.extend(SubmitPMIDList([i], Format, Bioconcept))

In [ ]:
res_json

In [ ]:
#print([res_json['cars'][i]['model'] for i in range(len(x['cars']))])
f = open('Pubtator_entities.txt', mode='w')

[f.write(res_json[i]['text'] + '\n') for i in range(len(res_json)) if(res_json[i]['infons']['type'] == 'Gene' or res_json[i]['infons']['type'] == 'Chemical')]
f.close()
f = open('Pubtator_entities.txt', mode='r')
print(f.read())
'''
for i in range(len(res_json)):
  if(res_json[i]['infons']['type'] == 'Gene' or res_json[i]['infons']['type'] == 'Chemical'):
    print(res_json[i]['text'], '\n')
'''


In [ ]:
input_text = 'Amyotrophic lateral sclerosis (ALS) is a progressive neurodegenerative disease that affects motor neurons. Mutations in the SPTLC1 subunit of serine palmitoyltransferase (SPT), which catalyzes the first step in the de novo synthesis of sphingolipids (SLs), cause childhood-onset ALS. SPTLC1-ALS variants map to a transmembrane domain that interacts with ORMDL proteins, negative regulators of SPT activity. We show that ORMDL binding to the holoenzyme complex is impaired in cells expressing pathogenic SPTLC1-ALS alleles, resulting in increased SL synthesis and a distinct lipid signature. C-terminal SPTLC1 variants cause peripheral hereditary sensory and autonomic neuropathy type 1 (HSAN1) due to the synthesis of 1-deoxysphingolipids (1-deoxySLs) that form when SPT metabolizes L-alanine instead of L-serine. Limiting L-serine availability in SPTLC1-ALS-expressing cells increased 1-deoxySL and shifted the SL profile from an ALS to an HSAN1-like signature. This effect was corroborated in an SPTLC1-ALS pedigree in which the index patient uniquely presented with an HSAN1 phenotype, increased 1-deoxySL levels, and an L-serine deficiency. These data demonstrate how pathogenic variants in different domains of SPTLC1 give rise to distinct clinical presentations that are nonetheless modifiable by substrate availability.'

In [ ]:
nlp = spacy.load("en_core_sci_sm") #instantiate scispacy model
doc = nlp(input_text)
print(doc.ents) # show entities created by the model
#for value, rel_dict in doc._.rel.items():
#    print(f"{value}: {rel_dict}")

In [ ]:
DEVICE = -1 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_sci_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe(
    "xx_coref", config={"chunk_size": 2500, "chunk_overlap": 2, "device": DEVICE})


# Define rel extraction model
rel_ext = spacy.load('en_core_sci_sm', disable=['ner', 'lemmatizer', 'attribute_rules', 'tagger'])
rel_ext.add_pipe("rebel", config={
    'device':DEVICE, # Number of the GPU, -1 if want to use CPU
    'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
    )

In [ ]:
coref_text = coref(input_text)._.resolved_text

doc = rel_ext(coref_text)

for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

In [ ]:
for value, rel_dict in doc._.rel.items():
  if ((rel_dict['head_span']['text'] in pub_ents) and (rel_dict['tail_span']['text'] in pub_ents)):
    cleaned_rel.append({rel_dict})